In [1]:
from everything import *

with open("music.pk", "rb") as f:
    music = pickle.load(f)

Using TensorFlow backend.


### Classifying composer by music snippet

First concatentate the arrays for each composer and split into manageable 256 length chunks

In [2]:
import itertools

groups = lambda: itertools.groupby(music, lambda x: x["composer"])
from scipy.sparse import vstack

block_per_composer = (
    (composer, vstack(x["piece"] for x in pieces).tocsr()) for composer, pieces in groups()
)

chunk_size = 1024

In [3]:
data = itertools.chain([
    (i[0], i[1][j : j + chunk_size])
    for j in range(0, i[1].shape[0], chunk_size)]
    for i in block_per_composer
)
final = list(itertools.chain(*data))

Remove any wrongly sized chunks

In [4]:
final = [i for i in final if i[1].shape==(chunk_size,128)]
import random 
random.shuffle(final)
y = [i[0] for i in final]

Lets listen to some music! See if we can guess correctly. Using pygame here to play the music!

In [139]:
x=Counter(i[0] for i in final)
num_comp = len(x)
print(music[0])
dict(sorted(((i,j) for i,j in x.items()), key=lambda x:x[1], reverse=True))

{'name': 'albeniz_suite_espanola_47_02_cataluna_(c)yogore.mid', 'composer': 'Albeniz', 'piece': <7368x128 sparse matrix of type '<class 'numpy.float64'>'
	with 28387 stored elements in COOrdinate format>}


{'Bach, J.S': 13512,
 'Mozart': 9147,
 'Beethoven': 6583,
 'Schubert': 6112,
 'Haydn, F': 3389,
 'Tchaikovsky': 3386,
 'Chopin': 3267,
 'Scarlatti, D': 3149,
 'Brahms': 3071,
 'Handel': 3026,
 'Alkan': 2774,
 'Dvořák': 2367,
 'Schumann': 1903,
 'Saint-Saëns': 1387,
 'Debussy': 1345,
 'Byrd': 1260,
 'Mahler': 1042,
 'Clementi': 983,
 'Dandrieu': 979,
 'Mendelssohn-B': 963,
 'Frescobaldi': 921,
 'Fauré': 810,
 'Pachelbel, J': 800,
 'Albeniz': 786,
 'Ravel': 671,
 'Buxtehude': 495,
 'Rachmaninov': 462,
 'Mussorgsky': 442,
 'Stravinsky': 408,
 'Froberger': 380,
 'Telemann': 342,
 'Satie': 337,
 'Vivaldi': 265,
 'Reger': 199,
 'Gesualdo da Venosa': 195,
 'Bartók': 195,
 'Pres (Despres)': 145,
 'Scriabin': 125,
 'Praetorius': 116,
 'Couperin': 4}

In [6]:
import music as mus
import pygame
import os
def play(piece):
    pygame.init()
    name = f"tester.mid"
    write_midi(piece.toarray()*127, name)
    pygame.mixer.music.load(name)
    pygame.mixer.music.play()
    os.remove(name)
    
def play_snippet(comp):
    pygame.init()
    name = f"tester.mid"
    snippet = random.choice([i for i in final if i[0]==comp])[1]
    mus.write_midi(snippet.toarray()*127, name)
    pygame.mixer.music.load(name)
    pygame.mixer.music.play()
    

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
play_snippet("Bach, J.S")

In [8]:
def test():
    answers = []
    for i, (comp, snippet) in enumerate(final[:50]):
        name = f"tester.mid"
        mus.write_midi(snippet.toarray()*127, name)
        pygame.mixer.music.load(name)
        pygame.mixer.music.play()
        inp = input("Who was it?")
        if len(inp)>=3 and inp in comp:
            print(f"Correct the composer was {comp}")
            answers.append((comp, True))
        else:
            print(f"Incorrect the composer was {comp}")
            answers.append((comp, False))
    

Split into train and test 

In [9]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y= le.fit_transform(y)
X = [i[1] for i in final]
n=len(y)
test_size = n//5
X_train, y_train = X[test_size:], y[test_size:]
X_test, y_test = X[:test_size], y[:test_size]

We will to need make generator to make sure they are passed as right format but not run out of memory

In [10]:
def to_batchs(x, y, batch_size ):
    xi = iter(x)
    yi = iter(y)
    while True:
        try:
            n = (np.array([np.array(next(xi).toarray()) for i in range(batch_size)]),np.array([next(yi) for i in range(batch_size)]))
        except StopIteration:
                 xi = iter(x)
                 yi = iter(y)
            
        yield n

Now let us see the results of a standard nn in classifing this

In [11]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(256, 128)))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(num_comp, activation=tf.nn.softmax))
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [12]:
#model.fit_generator(to_batchs(X_train, y_train, 1024), epochs=3, steps_per_epoch=1000)

In [13]:
#model.evaluate(to_batchs(X_test,y_test, 32), steps=400)

This is not very spectacular. As we can see below it only has 25% accuracy when our best guess at mozart is 14%! The high accuracy in training also indicateds it is highly overfit.

In [14]:
from collections import Counter
sorted(((i,(round(100*j/len(y),2))) for i,j in Counter(le.inverse_transform(y)).items()), key=lambda x:x[1], reverse=True)[:10]


[('Bach, J.S', 17.38),
 ('Mozart', 11.77),
 ('Beethoven', 8.47),
 ('Schubert', 7.86),
 ('Haydn, F', 4.36),
 ('Tchaikovsky', 4.36),
 ('Chopin', 4.2),
 ('Scarlatti, D', 4.05),
 ('Brahms', 3.95),
 ('Handel', 3.89)]

#### Lets try a convolutional network now.

In [15]:
def to_conv_batchs(x, y, batch_size ):
    xi = iter(x)
    yi = iter(y)
    while True:
        try:
            xb = np.array([np.array(next(xi).toarray()).reshape(chunk_size,128,1) for i in range(batch_size)])
            yb = np.array([next(yi)for i in range(batch_size)])
            n = (xb,yb)
        except StopIteration:
                 xi = iter(x)
                 yi = iter(y)
            
        yield n
        
#x = to_conv_batchs(X_train, y_train, 32)

In [299]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D,Dropout
#create model
model = Sequential()
#add model layers
model.add(Conv2D(32, kernel_size=(4,4), activation="relu", input_shape=(chunk_size, 128,1)))
model.add(Conv2D(16, kernel_size=(4,4), activation="relu"))
model.add(Conv2D(8, kernel_size=(4,4), activation="relu"))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(num_comp, activation="softmax"))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [300]:
model.fit_generator(to_conv_batchs(X_train, y_train, 8), epochs=6, steps_per_epoch=100)

Epoch 1/6


ResourceExhaustedError: OOM when allocating tensor with shape[4,4,32,16] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node training_3/Adam/Variable_10/Assign (defined at C:\Users\jembi\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:402)  = Assign[T=DT_FLOAT, _grappler_relax_allocator_constraints=true, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_3/Adam/Variable_10, training_3/Adam/zeros_10)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_3/Adam/Variable_10/Assign', defined at:
  File "C:\Users\jembi\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\jembi\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\jembi\Anaconda3\lib\site-packages\ipykernel

In [21]:
model.evaluate_generator(to_conv_batchs(X_test,y_test, 32), steps=400)

[2.524674486517906, 0.300859375]

In [24]:
#model = keras.models.load_model("conv-classical")

In [295]:
import pypianoroll
def write_midi(arr, filename):
    pypianoroll.Multitrack(tracks=[pypianoroll.Track(arr)]).write(filename)
n = 169
piece_s, composer = X_test[n], y_test[n]
piece = np.expand_dims(piece_s.toarray(), axis=3)
piece = np.expand_dims(piece, axis=0)
composer = np.expand_dims(composer, axis=0)
print(piece.shape)
pred = model.predict(piece)[0]
comp_num = np.argmax(pred)
print("PREDICTED ",le.classes_[comp_num]," ACTUAL ", le.classes_[composer[0]])
play(piece_s)
print(max(pred))
print(pred)

C:\Users\jembi\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  


(1, 1024, 128, 1)
PREDICTED  Mozart  ACTUAL  Alkan
0.21263775
[6.75711734e-03 2.77162697e-02 6.61955997e-02 7.39755225e-04
 1.00456811e-01 5.05469218e-02 2.74099963e-04 8.41391593e-05
 5.73112406e-02 9.70564224e-03 1.84483797e-04 6.86509360e-04
 1.74814742e-02 3.57328802e-02 6.50345720e-03 2.71426106e-04
 1.29963199e-04 1.60702431e-04 3.32118524e-03 3.98032889e-02
 1.43419229e-03 8.09417758e-03 2.12637752e-01 1.43039366e-03
 2.70365359e-04 3.35782417e-04 2.75095081e-04 1.54376845e-03
 7.27289007e-04 1.84798019e-03 1.78480726e-02 4.80968243e-04
 2.55208388e-02 1.86049476e-01 7.87518267e-03 3.61602579e-04
 6.56327838e-03 1.01428695e-01 7.90871622e-04 4.21171135e-04]


In [211]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1021, 125, 64)     1088      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1018, 122, 32)     32800     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3974272)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                158970920 
Total params: 159,004,808
Trainable params: 159,004,808
Non-trainable params: 0
_________________________________________________________________


In [219]:
def num_dups(mus):
    encoutered = set(pickle.dumps(i["piece"]) for i in mus)            
    return len(mus)-len(encoutered)
num_dups(music)

38

56